# Problem Statement
In this assignment we are using the MovieLens 1M dataset which contains 1000208 ratings given to about 3706 movies by 6040 users. Our aim is to implement three recommendation algorithms to estimate the rating a given user would give to a given movie. We will evaluate the accuracy of these algorithms with the Root Mean Squared Error (RMSE), and the Mean Absolute Error (MAE). We are using 5-fold cross-validation and report the RMSE and MAE on average of 5-folds for test dataset to obtain reliable results.  
The three algorithmes we implemented are:  
1. Naive Approaches
2. The UV matrix decomposition
3. The Matrix Factorization   

We will dicuss details of each algorithms and its implementation in three parts in this notebook.

It may happen that some users or movies that appear in a test set would not appear in the training set, we handled this situation on each algorithm differently which we will dicuss it later.

## Importing requirments and preparing dataset

In [2]:
# Downloading and preparing the dataset
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2023-10-22 19:28:17--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  10.6MB/s    in 0.5s    

2023-10-22 19:28:18 (10.6 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [3]:
# Importing requirments
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import copy
import os

### RATINGS FILE DESCRIPTION

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

In [4]:
# Reading ratings.dat file which contains all given ratings
df = pd.read_csv('/content/ml-1m/ratings.dat', sep="::", header=0, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
df.head(10)

<ipython-input-4-05e5c6dcba11>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/ml-1m/ratings.dat', sep="::", header=0, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])


,UserID,MovieID,Rating,Timestamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268
5,1,1287,5,978302039
6,1,2804,5,978300719
7,1,594,4,978302268
8,1,919,4,978301368
9,1,595,5,978824268


# 1. Naive Approaches







## Linear combination of the three averages:

## Description of algorithm:  
In this algorithm we use the global average rating, the average rating per item noted as R_item(User, Item) and the average rating per user noted as R_user(User, Item).
The goal is to estimate the rating for a given “active user” to the given item, R(User, Item).

We can estimate the rating using four different naive aproach:
  1. Using global average:  
  We just estimate the average of all ratings for any given user and movie
  2. Using user average:  
  We estimate the rating of a given user to a given movie based on all other average ratings that the user had given in our dataset.
  3. Using movie average:  
  We estimate the rating of a given user to a given movie based on all other average ratings that the movie had recieved based on all available data in our dataset.
  4. linear combination of the three averages:  
  In this approuch we approximate the R(User, Item) as a linear combination of R_item(User, Item) and R_user(User, Item). Using the following formula:  
  R(User, Item)= α * R_user(User, Item) + β * R_item(User, Item) + γ.  
  For finding “optimal” values for α, β and γ we can solve this problem as a least squares regression problem.  

## Details of implementation:  

For implementing 5-fold cross validation we used **KFold module from sklearn** which gives us indices of samples in train and test dataset for each fold. We are using a loop in which we iterate over five folds of dataset and in each iteration we train and evaluate a model with our naive approach. In each of approaches we calculate the average values which are needed. After that we enter the evaluation phase in which we calculate predicted values. After calculating the predicted values for each test data sample, we need to do a post process on results. For this porpus we wrote a function **post_process(results)** which round each estimated rating to the nearest integer and also keep results in a range of [1,5].   
After doing the post process, we used **mean_squared_error()** and **mean_absolute_error()** functions from sklearn to calculate RMSE amd MAE.

Finally we report the RMSE and MAE for each model on each fold along with the final RMSE and MAE on total of 5 folds.

***Handling gaps in numbering:*** In this algorithm the numbering of userIDs and movieIDs does not influence any calculation, so there is no need to to something about it. We can do renumbering as we did for matrix factorization part, but it will have no influence on final results of the algorithm.

***Handling unseen data:*** We are using average of the overal ratings as a fall back for all four approaches. So whenever it comes to an unseen user or an unssen movie we just use the average of all ratings in prediction formula.


### linear combination of the three averages:

#### **Some implementation details:**
For implementation of this algorithm we first need to calculate R_user(User, Item) for each user and R_item(User, Item) for each movie, for this porpus we wrote a function **get_R(df, column)** which calculate the average rating per item, the average rating per user. When column = 'UserID' is passed to it, it returns R_user(User, Item) and a map of UserID to mean of ratings all ratings this user have given to different movies. In the same way when column = 'MovieID' is passed to it, It returns R_item(User, Item) and a map of MovieID to mean of ratings all ratings this movie have gotten from all users.

In training phase we calculate R_item(User, Item) and R_user(User, Item) on train dataset. After that we form the linear equation using **np.vstack([R_user_train, R_item_train, np.ones(R_item_train.shape)]).T** then we use **np.linalg.lstsq()** to find optimal values for α, β, and γ and finish the training.  

As we find our three optimal parameters we can start the evaluation phase. In this part we predict a rating for each given user and movie in test dataset using our trained parameters and the linear formula. For this calculation we need to calculate average values R_user(User, Item) and R_item(User, Item) on test dataset. Then we use these average values along with trained parameters to calculate predicted value using the linear fomula of this algorithm.

In [5]:
# A function for calculating R_item(User, Item) and R_user(User, Item)
def get_R(df, column):
  R_map = {}
  for user in np.unique(df[column]):
    R_map[user] = np.mean(df[df[column] == user]['Rating'])
  R = np.array([R_map[df[column][i]] for i in df.index])

  return R, R_map

In [6]:
# A function for post processing
def post_process(results):
  return np.clip(np.round(results), 1, 5)

In [11]:
### Train and Evaluation ###

# Implementing 5-fold cross validation
number_of_folds = 5
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

rmse_list = []
mae_list = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):

  train_data = df.iloc[train_idx]
  test_data = df.iloc[test_idx]

  overall_train_rating = train_data['Rating'].mean()

  ### Train phase ###
  R_user_train, R_user_map_train = get_R(train_data, 'UserID')
  R_item_train, R_item_map_train = get_R(train_data, 'MovieID')

  # using np.linalg.lstsq() to solve this “least squares regression problem”
  A = np.vstack([R_user_train, R_item_train, np.ones(R_item_train.shape)]).T
  alpha, beta, gamma = np.linalg.lstsq(A, train_data['Rating'], rcond=None)[0]

  ### Evaluation phase ###
  # predict rating based on this formula:  alpha * R_user(User, Item) + beta * R_item(User, Item) + gamma
  predicteds = [alpha * R_user_map_train.get(row.UserID, overall_train_rating) + beta * R_item_map_train.get(row.MovieID, overall_train_rating) + gamma for row in test_data.itertuples()]
  predicteds_normalized = post_process(predicteds)

  # calculate RMSE and MAE of final model on test data
  rmse = mean_squared_error(test_data['Rating'], predicteds_normalized)
  mae = mean_absolute_error(test_data['Rating'], predicteds_normalized)

  rmse_list.append(rmse)
  mae_list.append(mae)
  print(f'RMSE for fold #', fold_idx+1, ': ', rmse)
  print(f'MAE for fold #', fold_idx+1, ': ', mae)

### Reporting the average error of five models ###
print(f'Mean RMSE on all 5 folds: ', np.mean(rmse_list))
print(f'Mean MAE on all 5 folds: ', np.mean(mae_list))

RMSE for fold # 1 :  0.9371981883804401
MAE for fold # 1 :  0.6948590795932854
RMSE for fold # 2 :  0.9345737395147019
MAE for fold # 2 :  0.6944641625258696
RMSE for fold # 3 :  0.9327141300326931
MAE for fold # 3 :  0.6944241709241059
RMSE for fold # 4 :  0.936667983063472
MAE for fold # 4 :  0.6963772426652536
RMSE for fold # 5 :  0.937707769907169
MAE for fold # 5 :  0.6964372303677746
Mean RMSE on all 5 folds:  0.9357723621796952
Mean MAE on all 5 folds:  0.6953123772152578


### Using movie average:

In [8]:
### Train and Evaluation ###

# Implementing 5-fold cross validation
number_of_folds = 5
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

rmse_list = []
mae_list = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):

  train_data = df.iloc[train_idx]
  test_data = df.iloc[test_idx]

  grouped_rating = train_data.groupby('MovieID')['Rating'].mean()
  overall_train_rating = train_data['Rating'].mean()

  ### No Train phase needed ###

  ### Evaluation phase ###

  # predict rating based on movie's average already rated ratings
  predicteds = [grouped_rating.get(row.MovieID, overall_train_rating) for row in test_data.itertuples()]
  predicteds_normalized = post_process(predicteds)

  # calculate RMSE and MAE of final model on test data
  rmse = mean_squared_error(test_data['Rating'], predicteds_normalized)
  mae = mean_absolute_error(test_data['Rating'], predicteds_normalized)

  rmse_list.append(rmse)
  mae_list.append(mae)
  print(f'RMSE for fold #', fold_idx+1, ': ', rmse)
  print(f'MAE for fold #', fold_idx+1, ': ', mae)

### Reporting the average error of five models ###
print(f'Mean RMSE on all 5 folds: ', np.mean(rmse_list))
print(f'Mean MAE on all 5 folds: ', np.mean(mae_list))

RMSE for fold # 1 :  1.0368722568260664
MAE for fold # 1 :  0.7510722748222873
RMSE for fold # 2 :  1.0377720678657483
MAE for fold # 2 :  0.7505723798002419
RMSE for fold # 3 :  1.0369672368802552
MAE for fold # 3 :  0.7508773157636897
RMSE for fold # 4 :  1.0377522607865388
MAE for fold # 4 :  0.7501312230992646
RMSE for fold # 5 :  1.0390919861428407
MAE for fold # 5 :  0.7517108992656505
Mean RMSE on all 5 folds:  1.03769116170029
Mean MAE on all 5 folds:  0.7508728185502268


### Using user average:

In [9]:
### Train and Evaluation ###

# Implementing 5-fold cross validation
number_of_folds = 5
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

rmse_list = []
mae_list = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):

  train_data = df.iloc[train_idx]
  test_data = df.iloc[test_idx]

  grouped_rating = train_data.groupby('UserID')['Rating'].mean()
  overall_train_rating = train_data['Rating'].mean()

  ### No Train phase needed ###

  ### Evaluation phase ###

  # predict rating based on user's average already rated ratings
  predicteds = [grouped_rating.get(row.UserID, overall_train_rating) for row in test_data.itertuples()]
  predicteds_normalized = post_process(predicteds)

  # calculate RMSE and MAE of final model on test data
  rmse = mean_squared_error(test_data['Rating'], predicteds_normalized)
  mae = mean_absolute_error(test_data['Rating'], predicteds_normalized)

  rmse_list.append(rmse)
  mae_list.append(mae)
  print(f'RMSE for fold #', fold_idx+1, ': ', rmse)
  print(f'MAE for fold #', fold_idx+1, ': ', mae)

### Reporting the average error of five models ###
print(f'Mean RMSE on all 5 folds: ', np.mean(rmse_list))
print(f'Mean MAE on all 5 folds: ', np.mean(mae_list))

RMSE for fold # 1 :  1.1506383659431518
MAE for fold # 1 :  0.7925935553533758
RMSE for fold # 2 :  1.1563971565971145
MAE for fold # 2 :  0.79451315223803
RMSE for fold # 3 :  1.1519730856520132
MAE for fold # 3 :  0.7930484598234371
RMSE for fold # 4 :  1.1563429496953124
MAE for fold # 4 :  0.7952869661719347
RMSE for fold # 5 :  1.1582575572007738
MAE for fold # 5 :  0.7963617458421024
Mean RMSE on all 5 folds:  1.1547218230176732
Mean MAE on all 5 folds:  0.794360775885776


### Using global average:

In [10]:
### Train and Evaluation ###

# Implementing 5-fold cross validation
number_of_folds = 5
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

rmse_list = []
mae_list = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):

  train_data = df.iloc[train_idx]
  test_data = df.iloc[test_idx]

  overall_train_rating = train_data['Rating'].mean()

  ### No Train phase needed ###

  ### Evaluation phase ###

  # predict rating based on overall training rating
  predicteds = [overall_train_rating for row in test_data.itertuples()]
  predicteds_normalized = post_process(predicteds)

  # calculate RMSE and MAE of final model on test data
  rmse = mean_squared_error(test_data['Rating'], predicteds_normalized)
  mae = mean_absolute_error(test_data['Rating'], predicteds_normalized)

  rmse_list.append(rmse)
  mae_list.append(mae)
  print(f'RMSE for fold #', fold_idx+1, ': ', rmse)
  print(f'MAE for fold #', fold_idx+1, ': ', mae)

### Reporting the average error of five models ###
print(f'Mean RMSE on all 5 folds: ', np.mean(rmse_list))
print(f'Mean MAE on all 5 folds: ', np.mean(mae_list))

RMSE for fold # 1 :  1.4251957089011307
MAE for fold # 1 :  0.8714719908819148
RMSE for fold # 2 :  1.4224262904789995
MAE for fold # 2 :  0.8707621399506104
RMSE for fold # 3 :  1.4189320242749022
MAE for fold # 3 :  0.8693274412373402
RMSE for fold # 4 :  1.4254277873036028
MAE for fold # 4 :  0.8719612479441714
RMSE for fold # 5 :  1.4230382771531835
MAE for fold # 5 :  0.8712813873156003
Mean RMSE on all 5 folds:  1.4230040176223635
Mean MAE on all 5 folds:  0.8709608414659276


##Conclusion:  
The final results are:   
 1. Using global average:  
    Mean RMSE on all 5 folds:  1.4230040176223635  
    Mean MAE on all 5 folds:  0.8709608414659276

 2. Using user average:  
 Mean RMSE on all 5 folds:  1.1547218230176732   
Mean MAE on all 5 folds:  0.794360775885776

3. Using movie average:    
Mean RMSE on all 5 folds:  1.03769116170029  
Mean MAE on all 5 folds:  0.7508728185502268  

4. linear combination of the three averages:  
Mean RMSE on all 5 folds:  0.8935461434633012  
Mean MAE on all 5 folds:  0.6745077030556752   


We observe that the global average has the worst performance among other naive approaches. Then using user average has a better and lower RMSE compare to the movie average method. And the best performance is for the linear combination algorithm. The linear combination algorithm achieve a somehow acceptable performance with a relatively low runtime cost. It took about one minute to train all 5 models.

# 2. The UV matrix decomposition
We have implemented UV decomposition according to chapter 9.4 of MMD. The base idea is that we search for sparse matrices $U$ and $V$ so as to minimize the mean squared error of $M - UV$ (for known values). To this end, we iterate through each element of $U$ and $V$ and set it to the optimal value to minimise the MSE relative to all other current values of $U$ and $V$.

In the base train iteration we choose to iterate through all encoding elements of U and V in simple $((U, V), row, column)$ ordering since we have no indication that any optimization ordering biases the training in any way.

For the encoding size we have chosen $d = 10$ since we consider 10 a reasonable estimation of pertinent movie tags which may describe genre-like linearly combinable characteristics (e.g. violent, feel-good, cerebral).

For preprocessing we choose to subtract from the rating matrix the average scores per users and then the average scores per movies.

We chose to initialize matrices $U$ and $V$ with random values normally distributed with mean 0 and standard deviation $\sqrt{\sigma(M)}$. The instructions in MMD reccomend using a mean value of $\mu(M)$ however we argue that this recommendation is not compatible with the preprocessing procedure recommended, specifically because, after preprocessing, the values in $M$ become arbitrarily positive or negative, and thus the encoding parameters also need to be arbitrarily positive or negative, and any judgement about the expected product of two encodings is not applicable.

For postprocessing we clip the predicted values to $[1, 5]$ and round them to the nearest integer. We note that none of the preprocessing and postprocessing steps improved the performance of the algorithm, but all of them collectively significantly slowed down its runtime.

***Handling unseen data:*** For unseen users or movies we use the average encoding over users in U or movies in V, respectively.

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt
signed_sqrt = lambda x: sqrt(x) if x >= 0 else -sqrt(-x)
root_mean_squared_error = lambda *args, **kwargs: sqrt(mean_squared_error(*args, **kwargs))

def relative_diff(a, b):
  return 2 * abs(a - b) / abs(a + b)

def cond_string(cond, string):
  if not cond:
    return ''
  return string()

def post_process(a):
  return np.clip(np.round(a), 1, 5)

class UVDecomposition:
  def init_and_get_train_object(s, df, d=10, preprocessing=True, randomness_factor=1):
    fix_init_value = not preprocessing

    M = df.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=np.NAN)
    s.movie_pos = {val: i for val, i in zip(M.columns, range(len(M.columns)))}
    s.user_pos = {val: i for val, i in zip(M.index, range(len(M.index)))}

    M = np.array(M)
    s.no_users, s.no_movies = M.shape

    s.d = d

    if preprocessing:
      s.user_means = np.nanmean(M, axis=1, keepdims=True)
      s.user_mean = np.nanmean(s.user_means)
      M -= s.user_means
      s.movie_means = np.nanmean(M, axis=0, keepdims=True)
      s.movie_mean = np.nanmean(s.movie_means)
      M -= s.movie_means
    else:
      s.user_means = np.zeros((M.shape[0], 1))
      s.user_mean = 0
      s.movie_means = np.zeros((1, M.shape[1]))
      s.movie_mean = 0

    if fix_init_value:
      init_value = sqrt(abs(np.nanmean(M)) / d)
    else:
      init_value = 0
    init_std = randomness_factor * sqrt(np.nanstd(M) / d)
    s.U = init_value + np.random.normal(scale=init_std, size=(s.no_users, s.d))
    s.V = init_value + np.random.normal(scale=init_std, size=(s.d, s.no_movies))

    s.epoch = 0
    s.train_err = []
    s.val_err = []
    s.user_mean_encoding = np.mean(s.U, axis=0)
    s.movie_mean_encoding = np.mean(s.V, axis=1)
    return M

  def train_epoch(s, M, verbose=False, val=None):
    s.epoch += 1

    for r in range(s.no_users):
      for si in range(s.d):
        s.U[r, si] += np.nansum(s.V[si] * (M[r] - s.U[r] @ s.V)) / np.sum(s.V[si] ** 2)

    for r in range(s.d):
      for si in range(s.no_movies):
        s.V[r, si] += np.nansum(s.U[:, r] * (M[:, si] - s.U @ s.V[:, si])) / np.sum(s.U[:, r] ** 2)

    s.user_mean_encoding = np.mean(s.U, axis=0)
    s.movie_mean_encoding = np.mean(s.V, axis=1)

    s.train_err.append(s.train_rmse(M))
    if val:
      s.val_err.append(s.val_rmse(val))

    if verbose:
      print(f'UVDecomposition: Epoch {s.epoch}, train RMSE {s.train_rmse(M)}' + cond_string(val, lambda: f', validation RMSE {s.val_rmse(val)}'))

  def fit(s, M, verbose=True, num_iter=None, val=None, stop_condition='convergence', skip_verbose_step=1):
    if verbose:
      print(f'UVDecomposition: Initialization, train RMSE {s.train_rmse(M)}' + cond_string(val, lambda: f', validation RMSE {s.val_rmse(val)}'))

    if num_iter:
      for _ in range(num_iter):
        s.train_epoch(M, verbose=verbose and not ((s.epoch + 1) % skip_verbose_step), val=val)
      return
    if stop_condition == 'convergence':
      while len(s.train_err) < 2 or (s.train_err[-1] < s.train_err[-2] and relative_diff(s.train_err[-1], s.train_err[-2]) >= 0.005):
        s.train_epoch(M, verbose=verbose and not ((s.epoch + 1) % skip_verbose_step), val=val)
      return
    if stop_condition == 'overfit':
      if val is None:
        raise ValueError('Stop condition "overfit" called without validation set')
      while len(s.val_err) < 2 or s.val_err[-1] < s.val_err[-2] or relative_diff(s.val_err[-1], s.val_err[-2]) < 0.01:
        s.train_epoch(M, verbose=verbose and not ((s.epoch + 1) % skip_verbose_step), val=val)
      return
    raise ValueError(f'Unknown stop condition "{stop_condition}"')

  def predict(s, users, movies):
    return np.array([s.predict_single(user, movie) for user, movie in zip(users, movies)])

  def predict_single(s, user, movie):
    user_encoding = s.U[s.user_pos[user]] if user in s.user_pos else s.user_mean_encoding
    user_mean = s.user_means[s.user_pos[user]] if user in s.user_pos else s.user_mean
    movie_encoding = s.V[:, s.movie_pos[movie]] if movie in s.movie_pos else s.movie_mean_encoding
    movie_mean = s.movie_means[:, s.movie_pos[movie]] if movie in s.movie_pos else s.movie_mean
    return post_process(user_encoding @ movie_encoding + user_mean + movie_mean)

  def train_rmse(s, M):
    return np.nanmean((M + s.user_means + s.movie_means - post_process(s.U @ s.V + s.user_means + s.movie_means)) ** 2) ** (1 / 2)

  def val_rmse(s, val):
    (user_val, movie_val), rating_val = val
    return root_mean_squared_error(s.predict(user_val, movie_val), rating_val)

# 5-fold cross-validation
For parity with the experiment in Task 3, we have chosen to run each instance of the algorithm for 75 epochs.

In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

number_of_folds = 5
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

def split_dataframe(df):
  users = df['UserID']
  movies = df['MovieID']
  ratings = df['Rating']
  return (users, movies), ratings

def experiment(single=False, omit_validation=False, **params):
  abse_list = []
  rmse_list = []
  for fold_idx, (train_idx, val_idx) in enumerate(kf.split(df)):
    model = UVDecomposition()

    M_train = model.init_and_get_train_object(df.iloc[train_idx], **params)
    (user_val, movie_val), rating_val = split_dataframe(df.iloc[val_idx])

    if omit_validation:
        val = None
    else:
        val = (user_val, movie_val), rating_val
    
    model.fit(M_train, verbose=True, val=val, num_iter=75, skip_verbose_step=10)

    val_predict = model.predict(user_val, movie_val)

    rmse = root_mean_squared_error(val_predict, rating_val)
    rmse_list.append(rmse)
    print(f'RMSE for fold #', fold_idx+1, ': ', rmse)

    abse = mean_absolute_error(val_predict, rating_val)
    abse_list.append(abse)
    print(f'ABSE for fold #', fold_idx+1, ': ', abse)

    if single:
      return

  print(f'Mean RMSE: ', np.mean(rmse_list))
  print(f'Mean ABSE: ', np.mean(abse_list))

In [ ]:
experiment(omit_validation=True, d=10, preprocessing=True)

UVDecomposition: Initialization, train RMSE 1.0048845545430323
UVDecomposition: Epoch 10, train RMSE 0.9506037519098967
UVDecomposition: Epoch 20, train RMSE 0.9173918826585118
UVDecomposition: Epoch 30, train RMSE 0.8910718226573685
UVDecomposition: Epoch 40, train RMSE 0.8743598316417551
UVDecomposition: Epoch 50, train RMSE 0.8633845161278328
UVDecomposition: Epoch 60, train RMSE 0.8550041291405406
UVDecomposition: Epoch 70, train RMSE 0.8491630446154963
UVDecomposition: Epoch 75, train RMSE 0.8466824961080675
RMSE for fold # 1 :  0.9376248612958654
ABSE for fold # 1 :  0.6526579418322153
UVDecomposition: Initialization, train RMSE 1.0045524408594428
UVDecomposition: Epoch 10, train RMSE 0.9490880310177745
UVDecomposition: Epoch 20, train RMSE 0.9134826771757573
UVDecomposition: Epoch 30, train RMSE 0.88739948633435
UVDecomposition: Epoch 40, train RMSE 0.8716578136666325
UVDecomposition: Epoch 50, train RMSE 0.8607939646066463
UVDecomposition: Epoch 60, train RMSE 0.852954592698831

Our average RMSE is 0.93 and MAE 0.65.

# 3. The Matrix Factorization

## Description of algorithm:  
The idea behind matrix factorization (MF) techniques is that we want to approximate the matrix X as the product of two matrices: $ X ≃ UM $.

Where $ X $ is a matrix of ratings where rows represent userIDs and columns represent movieIDs. $ U $ is an I × K and $ M $ is a K × J matrix. The $ u_{ik} $ and $ m_{kj} $ values can be considered, reps. the kth feature of the ith user and the jth movie. Our goal in this algorithm is to find best values for each element in $ U $ and $ M $.  

In the case of our recommender system problem we want to approximate each unknown element in $ X $ using following formula:  
$\hat{x_{ij}} = \sum_{k=1}^{K} u_{ik}m_{kj} $.   
$ e_{ij} = x_{ij} - \hat{x_{ij}}$ .   
$ SE = \sum_{i,j \in R } e_{ij}$.

Here $ \hat{x_{ij}} $ denotes how the ith user would rate the jth movie, according to the model, eij denotes the training error on the (i, j)th example, and SE denotes the total squared training error. The optimal $ U $ and $ M $ minimizes the sum of squared errors only on the known elements of $ X $. In order to minimize SE (which is equivalent to minimize RMSE), we have applied a simple gradient descent method to find a local minimum.

We update the weights in the opposite direction of gradient using following formulas:

$ {u_{ik}}\prime = u_{ik} + η · (2e_{ij} · m_{kj} − λ · u_{ik}) $ and   
$ {m_{kj}}\prime = m_{kj} + η · (2e_{ij} · u_{ik} − λ · m_{kj}) $.

So this will be the steps of the algorithm:  
1. Initialize the weights in $ U $ and $ M $ randomly.
Set η and λ to some small positive value.

2. Loop until the terminal condition is met (The loop is terminated when the RMSE does not decrease
during two iterations.)
  - Iterate over each known element of $ X $ on train dataset
  - For each $ x_{ij} $, compute $ e_{ij} $
  - Update the ith row of $ U $ and the jth column of $ M $ according to equations mentioned for updates.  
  - After each iteration on train data, calculate RMSE and check terminal condition

3. We will save encoding matrices $ U $ and $ M $ after training of the model finished since we need these encodings for doing the visualizaions.

## Details of implementation:

In this implementation we renumber the overal dataset at first. Then we use **k-fold** from sklearn to find train and test indices for each fold. We repeat the following steps for 5 times in a loop for our cross validation:
 1. Initializing X, U and M. We initialize X by using a **df.pivot_table()** using userID as indices, movieID as columns and Ratings as values. We call df.pivot_table() on the whole dataset and fill values which should be in test set with NaN we also feel all empty elements with NaNs. Then we initialize U and M randomly based on the size of X using **np.random.rand()**.
 2. After initialization the main part of training gets start. We iterate on each row of train dataframe (x_ij), calculate product of U_ik and M_kj for that coresponding row and calculate error between result of this product and rating value in train sample. We update weights of U_ik and M_kj based on the error in this form:  
    U_matrix[index_in_U, :] = U_vector + learning_rate * (((2*error) * M_vector) - (regularization_factor * U_vector))

      M_matrix[:, index_in_M] = M_vector + learning_rate * (((2*error) * U_vector) - (regularization_factor * M_vector))

3. After one iteration on all rows of train dataset, we calculate RMSE, to check the termination condition. In our experiment we also set a **max_iteration_num = 75** to lessen the computation time. If the termination condition wasn't met, we repeat step 2. We print RMSE after each iteration to keep track of the training process and observe that at the end of each iteration RMSE decrease.

4. After termination of the training process, we start evaluation on testdataset. After calculating the predicted values for each test data sample, For this porpuse we wrote a **predict_rating(row)** function which we apply to the test data frame and get predicted results for each data sample(each row). Then we need to do a post process on results. For this porpus we wrote a function **post_process(results)** which round each estimated rating to the nearest integer and also keep results in a range of [1,5]. After doing the post process, we used **mean_squared_error()** and **mean_absolute_error()** functions from sklearn to calculate RMSE amd MAE.


***Handling gaps in numbering:*** For handling gaps in numbering on the official dataset, we wrote a function **renumbering(input_df, col)** which renumbers the input dataframe based on UserId and MovieID columns.

***Handling gaps in numbering which occurs during cross-validation on train or test dataset:*** These kind of gaps were making trouble in our algorithm since we make $U$ and $M$ matrices based on size and indices of users and movies in $X$. In these cases if we built $X$, $U$ and $M$ just based sampels in  the train dataset the order of indices on this matrices wouldn't mach the order of data samples' indices for test data. For this reason, we initialize $ X $ for training in size of the original dataset and just masked indices which supposed to be used as test data using NaNs (so we just replace NaN for ratings that have to remain unknown in train phase).


***Handling unseen data:*** For estimating rating for unseen userID we just return the average rating for the seen movieID, in the same way, for estimating rating for unseen movieID we just return the averahe for the seen userID. We are using a naive approach here, we can also use the approuch we used for handling unseen data in UV matrix decomposition and use the average user encodings for an unssen userID or average movie encoding for an unseen movieID.

In [ ]:
# Preprocess to eliminate gaps in numbering (by renumbering UserID and MovieID)
def renumbering(input_df, col):
  print('max_'+col+'_id: ',  max(df[col]))

  # Create a mapping of existing IDs to new IDs
  unique_ids = input_df[col].unique()
  new_ids = list(range(1, len(unique_ids) + 1))
  id_mapping = dict(zip(unique_ids, new_ids))
  reverse_id_mapping = dict([(value, key) for key, value in id_mapping.items()])

  # Replace the old IDs with the new IDs
  input_df[col] = input_df[col].map(id_mapping)

  print(f'{col} renumbered')
  print('max_'+col+'_id: ', max(df[col]))
  return input_df, reverse_id_mapping


# renumbering 'UserID' in case there is any gaps
df, userID_newtoold_mapping = renumbering(df, 'UserID')


# renumbering 'MovieID' in case there is any gaps
df, movieID_newtoold_mapping = renumbering(df, 'MovieID')

max_UserID_id:  6040
UserID renumbered
max_UserID_id:  6040
max_MovieID_id:  3952
MovieID renumbered
max_MovieID_id:  3706


In [ ]:
# A function for post processing
def post_process(results):
  return np.clip(np.round(results), 1, 5)

In [ ]:
### Setting hyperparameters ###
num_factor = 10
learning_rate = 0.005
regularization_factor = 0.05
max_num_iter = 75
number_of_folds = 5


# Implementing 5-fold cross validation
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=42)

train_indices_list = []
test_indices_list = []
rmse_list = []
mae_list = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):
  train_data = df.iloc[train_idx]
  train_users = [row.UserID for row in train_data.itertuples()]
  train_movies = [row.MovieID for row in train_data.itertuples()]
  test_data = df.iloc[test_idx]

  ### Train Phase ###
  t = copy.deepcopy(df)
  t.loc[test_idx, 'Rating'] = np.nan
  X_matrix = t.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=np.NAN, dropna = False)
  X_matrix = np.array(X_matrix)

  number_of_users, number_of_movies = X_matrix.shape

  # U dimension is I*K (k = num_factors, I = number of users)
  U_matrix = np.random.rand(number_of_users, num_factor)

  # M dimension is K*J (k = num_factors, I = number of movies)
  M_matrix = np.random.rand(num_factor, number_of_movies)

  prev_U_matrix = None
  prev_M_matrix = None

  current_RMSE = np.sqrt(np.nanmean((X_matrix - U_matrix @ M_matrix) ** 2))
  prev_RMSE = np.math.inf
  iter_num = 0

  # Train loop
  while (current_RMSE < prev_RMSE and iter_num < max_num_iter):
    prev_U_matrix = U_matrix
    prev_M_matrix = M_matrix
    for row in train_data.itertuples():
      index_in_U = row.UserID - 1
      index_in_M = row.MovieID - 1
      U_vector = U_matrix[index_in_U, :] #U_ik
      M_vector = M_matrix[:, index_in_M] #M_kj

      # approximated X_ij
      result = np.dot(U_vector, M_vector)

      # calculating error e_ij
      error = row.Rating - result

      # update the ith row of U and jth column of M according to error gradient and using a gradient decent approuch with regularization
      U_matrix[index_in_U, :] = U_vector + learning_rate * (((2*error) * M_vector) - (regularization_factor * U_vector))
      M_matrix[:, index_in_M] = M_vector + learning_rate * (((2*error) * U_vector) - (regularization_factor * M_vector))

    # calculating RMSE
    new_RMSE = np.sqrt(np.nanmean((X_matrix - post_process(U_matrix @ M_matrix)) ** 2))
    prev_RMSE = current_RMSE
    current_RMSE = new_RMSE
    iter_num += 1
    print(f'iteration {iter_num} in fold {fold_idx+1}, training rmse: {current_RMSE}')

  # train_RMSE = np.sqrt(np.nanmean((X_matrix - prev_U_matrix @ prev_M_matrix) ** 2))
  print(f'%%%%% Finished Training: RMSE on train data for fold #', fold_idx+1, ' : ', current_RMSE, '%%%%%')

  ### save trained encodings ###
  movieIDs = np.array([movieID_newtoold_mapping[index+1] for index in np.arange(number_of_movies)])
  userIDs = np.array([userID_newtoold_mapping[index+1] for index in np.arange(number_of_users)])

  np.save(os.path.join('/content/', 'user_encodings_MF_'+str(fold_idx+1)), np.concatenate((userIDs.reshape(-1, 1), prev_U_matrix), axis=1))
  np.save(os.path.join('/content/', 'movie_encodings_MF_'+str(fold_idx+1)), np.concatenate((movieIDs.reshape(-1, 1), np.transpose(prev_M_matrix)), axis=1))


  ### Test Phase ###
  predicted = []
  movieMean = train_data.groupby('MovieID')['Rating'].mean()
  userMean = train_data.groupby('UserID')['Rating'].mean()
  overall_mean = train_data['Rating'].mean()

  def predict_rating(row):
    if row['UserID'] in userMean and row['MovieID'] in movieMean:
      # Predict using your model
      index_in_U = row.UserID - 1
      index_in_M = row.MovieID - 1
      U_vector = U_matrix[index_in_U, :] #U_ik
      M_vector = M_matrix[:, index_in_M] #M_kj
      # approximated X_ij
      result = np.dot(U_vector, M_vector)
      return result
    elif row['UserID'] in userMean:
        # Predict using movie mean
        return movieMean.get(row['MovieID'], overall_mean)
    elif row['MovieID'] in movieMean:
        # Predict using user mean
        return userMean.get(row['UserID'], overall_mean)
    else:
        # Both user and movie are unseen, predict using overall mean
        return overall_mean

  output = test_data.apply(predict_rating, axis=1)

  normalized_output = post_process(output)

  test_RMSE = mean_squared_error(test_data['Rating'], normalized_output)
  test_MAE = mean_absolute_error(test_data['Rating'], normalized_output)

  print(f'%%%%% Finished Testing: RMSE on test data for fold #', fold_idx+1, ': ', test_RMSE, '%%%%%')
  print(f'%%%%% Finished Testing: MAE on test data for fold #', fold_idx+1, ': ', test_MAE, '%%%%%')

  rmse_list.append(test_RMSE)
  mae_list.append(test_MAE)

### Reporting the average error of five models ###
print(f'Mean RMSE on all 5 folds (on test data): ', np.mean(rmse_list))
print(f'Mean MAE on all 5 folds (on test data): ', np.mean(mae_list))

iteration 10 in fold 1, training rmse: 0.8974194322947476
iteration 20 in fold 1, training rmse: 0.8553913860959456
iteration 30 in fold 1, training rmse: 0.8392393002355556
iteration 40 in fold 1, training rmse: 0.8318264082436364
iteration 50 in fold 1, training rmse: 0.8276346517837789
iteration 60 in fold 1, training rmse: 0.8249219945051585
iteration 75 in fold 1, training rmse: 0.8226471447240427
%%%%% Finished Training: RMSE on train data for fold # 1  :  0.8226471447240427 %%%%%
%%%%% Finished Testing: RMSE on test data for fold # 1 :  0.8421681446896152 %%%%%
%%%%% Finished Testing: MAE on test data for fold # 1 :  0.6390408014316994 %%%%%
iteration 10 in fold 2, training rmse: 0.898030572851627
iteration 20 in fold 2, training rmse: 0.8564660189910339
iteration 30 in fold 2, training rmse: 0.8411803767513581
iteration 40 in fold 2, training rmse: 0.8337989815873477
iteration 50 in fold 2, training rmse: 0.829475577597213
iteration 60 in fold 2, training rmse: 0.82668657439282

##Conclusion:  
The final results are:   
Mean RMSE on all 5 folds:  0.84812859446636062.  
Mean MAE on all 5 folds:  0.6419894692476156.   
We observe that this algorithm achieve a better performance in comparision to the naive approach while it has a significantly longer runtime. It took about two hours to train and evaluate all 5 models.

# Final Conclusion:
The best performance is achieved by the Matrix Factorization algorithm, with a significant improvement to all other methods, RMSE 0.84, MAE 0.64.

The best of the naive approaches, the linear combination of averages, achieves competitive results, RMSE 0.89, MAE 0.67.

UV decomposition achieves better results than the first 3 naive approaches, but performs slightly worse than linear combination of averages and Matrix Factorization, with RMSE 0.93 and MAE 0.65. We suspect this is because the UV decomposition lacks any regularization mechanism, whereas MF includes explicit regularization, and the linear combination of averages has only 3 parameters. Supporting this idea we note that the train RMSE for UV decomposition is about 0.84, on par with the train performance of MF, 0.82.